In [0]:
from pyspark.sql import functions as F

In [0]:
fact_bike_ml = (spark.table("hive_metastore.tfm.fact_bike"))
dim_bike = (spark.table("hive_metastore.tfm.dim_bike"))
weather_station = spark.table('hive_metastore.tfm.fact_weather_enriched').alias('weather')
dim_calendar = (spark.table("hive_metastore.tfm.dim_calendar"))

fact_bike_ml.cache()
dim_bike.cache()
weather_station.cache()
dim_calendar.cache()

DataFrame[date: date, is_holiday: int, is_pre_holiday: int, is_weekend: int, day_of_week: int, month: int]

In [0]:
display(fact_bike_ml.limit(10))

started_date,bike_start_station_id,bike_end_station_id,total,distancia
2024-05-24,5297.02,6955.01,2,6.337510689899895
2024-05-19,6382.05,6955.01,1,2.3877275679440517
2024-05-24,8550.01,8811.01,1,2.5712472232605883
2024-05-16,7149.05,7084.12,1,2.9385806448563185
2024-05-17,7610.13,6925.09,1,3.43506560182014
2024-05-21,7981.27,7981.27,1,0.0
2024-05-24,6682.08,6732.01,1,1.1618477353499652
2024-05-29,7092.06,6948.10,1,2.065966409448513
2024-05-23,6492.08,6925.09,1,3.6985330207407747
2024-05-31,8615.01,8615.01,1,0.0


1. JOIN DIM BIKE

In [0]:
fact = spark.table('hive_metastore.tfm.fact_bike')
dim_bike = spark.table('hive_metastore.tfm.dim_bike').alias('dim')

fact_bike_ml = (
    fact.alias('fact')
    .join(
        dim_bike,
        F.col('fact.bike_start_station_id') == F.col('dim.bike_station_id'),
        'left'
    )
    .select(
        F.col('fact.started_date'),
        F.col('fact.bike_start_station_id'),
        F.col('fact.bike_end_station_id'),
        F.col('fact.total'),
        F.col('fact.distancia'),
        F.col('dim.weather_station_id').alias('weather_station_id')
    )
)

In [0]:
display(fact_bike_ml.limit(10))

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

2. Crear columna concatenada para el JOIN con la tabla de weather

In [0]:
fact_bike_ml = (
    fact_bike_ml
    .withColumn(
        'weather_station_date',
        F.concat_ws('/', F.col('weather_station_id'), F.col('started_date'))
    )
)

In [0]:
display(fact_bike_ml.limit(10))

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

3. Agrupar para reducir numero de filas

In [0]:
fact_bike_ml = (
    fact_bike_ml
    .groupBy(
        'started_date',
        'weather_station_date'
    )
    .agg(
        F.sum(F.col('total')).alias('total'),
        F.avg(F.col('distancia')).alias('avg_distancia')
    )
)

In [0]:
fact_bike_ml.count()

7017

In [0]:
fact_bike_ml.cache()

DataFrame[started_date: date, weather_station_date: string, total: bigint, avg_distancia: double]

In [0]:
display(fact_bike_ml.limit(10))

started_date,weather_station_date,total,avg_distancia
2024-05-29,USW00014734/2024-05-29,611,2.901460557216589
2024-09-07,USW00094728/2024-09-07,129173,2.400920106199063
2024-10-14,USW00094728/2024-10-14,120368,2.4713517271086394
2024-12-16,USW00014732/2024-12-16,4919,1.7996580877721484
2024-04-19,USW00094728/2024-04-19,105049,2.23592020906279
2024-12-09,2024-12-09,46,null
2024-10-26,USW00094789/2024-10-26,29,2.5365776004583642
2024-12-19,2024-12-19,30,null
2024-07-29,2024-07-29,87,null
2021-10-04,USW00014732/2021-10-04,3721,1.8355061697456985


3. JOIN WEATHER

In [0]:

fact = fact_bike_ml.alias('fact')
weather = weather_station.alias('weather')

fact_bike_ml = (
    fact
    .join(
        weather_station,
        on = 'weather_station_date',
        how = 'left'
    )
    .select(
        F.col('fact.started_date'),
        F.col('fact.total'),
        F.col('fact.avg_distancia'),

        F.col('weather.awnd_bin').alias('awnd_bin'),
        F.col('weather.prcp_bin').alias('prcp_bin'),
        F.col('weather.snow_bin').alias('snow_bin'),
        F.col('weather.snwd_bin').alias('snwd_bin'),
        F.col('weather.tmax_bin').alias('tmax_bin'),
        F.col('weather.tmin_bin').alias('tmin_bin'),
        F.col('weather.trange_bin').alias('trange_bin')
    ).where(F.year('started_date') >= 2020)
)

In [0]:
display(fact_bike_ml.limit(10))

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:728)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:446)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:446)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

4. JOIN HOLIDAY

In [0]:
fact = fact_bike_ml.alias('fact')
calendar = dim_calendar.alias('calendar')

fact_bike_ml = (
    fact
    .join(
        calendar,
        fact['started_date'] == calendar['date'],
        how='left'
    )
    .select(
        F.col('fact.started_date').alias('date'),
        F.col('fact.total'),
        F.col('fact.avg_distancia'),
        F.col('fact.awnd_bin'),
        F.col('fact.prcp_bin'),
        F.col('fact.snow_bin'),
        F.col('fact.snwd_bin'),
        F.col('fact.tmax_bin'),
        F.col('fact.tmin_bin'),
        F.col('fact.trange_bin'),
        F.col('calendar.is_holiday'),
        F.col('calendar.is_pre_holiday'),
        F.col('calendar.is_weekend'),
        F.col('calendar.day_of_week'),
        F.col('calendar.month')
    )
)

In [0]:
display(fact_bike_ml.limit(10))

date,total,avg_distancia,awnd_bin,prcp_bin,snow_bin,snwd_bin,tmax_bin,tmin_bin,trange_bin,is_holiday,is_pre_holiday,is_weekend,day_of_week,month
2024-05-29,611,2.901460557216589,cat3,cat0,cat0,cat0,cat4,cat3,cat5,0,0,0,4,5
2024-09-07,129173,2.400920106199063,cat0,cat2,cat0,cat0,cat0,cat0,cat2,0,0,1,7,9
2024-10-14,120368,2.4713517271086394,cat1,cat0,cat0,cat0,cat0,cat0,cat2,1,0,0,2,10
2024-12-16,4919,1.7996580877721484,cat4,cat1,cat0,cat0,cat2,cat1,cat3,0,0,0,2,12
2024-04-19,105049,2.23592020906279,cat1,cat0,cat0,cat0,cat0,cat0,cat1,0,0,0,6,4
2024-12-09,46,null,null,null,null,null,null,null,null,0,0,0,2,12
2024-10-26,29,2.5365776004583642,cat6,cat0,cat0,cat0,cat3,cat2,cat4,0,0,1,7,10
2024-12-19,30,null,null,null,null,null,null,null,null,0,0,0,5,12
2024-07-29,87,null,null,null,null,null,null,null,null,0,0,0,2,7
2021-10-04,3721,1.8355061697456985,cat4,cat0,cat0,cat0,cat4,cat3,cat2,0,0,0,2,10


5. Guardar tabla

In [0]:
%sql
DROP TABLE IF EXISTS tfm.fact_bike_ml

In [0]:
fact_bike_ml.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true").option("overwrite", "true") \
    .saveAsTable("tfm.fact_bike_ml")